In [1]:
# from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
def load_az_dataset(datasetpath):
    
    labels = []
    data = []
    
    for row in open(datasetpath):
        row = row.split(",")
        label = int(row[0])
        image = np.array([int(x) for x in row[1:]],dtype = "uint8")
        
        #reshaping the image in 28*28 matrix
        
        image = image.reshape((28,28))
        
        #updating data and lables
        
        data.append(image)
        labels.append(label)
        
    data = np.array(data,dtype="float32")
    labels = np.array(labels,dtype="int")
    
    return (data,labels)

In [4]:
def load_mnist_dataset():
    
    ((train_data,train_labels),(test_data,test_labels)) = mnist.load_data()
    
    data = np.vstack([train_data,test_data])
    labels = np.hstack([test_labels,train_labels])
    
    return (data,labels)


In [5]:
data_a_z ,label_a_z = load_az_dataset("../data/archive/a.csv")
data_mnist, label_mnist = load_mnist_dataset()
label_a_z += 10
    
data_total = np.vstack([data_mnist,data_a_z])
labels_total = np.hstack([label_mnist,label_a_z])



In [6]:
print(labels_total.shape)
print(data_total.shape)
print(label_a_z.shape)

(442451,)
(442451, 28, 28)
(372451,)


In [7]:
data_dim = np.expand_dims(data_total, axis=-1)
print(data_total.shape)
# data_total /= 255.0
data_dim = np.divide(data_dim,255.0)
(X_train ,X_test, y_train , y_test) = train_test_split(data_dim,labels_total, test_size=0.20, stratify=labels_total, random_state=42)
print(X_train.shape)

(442451, 28, 28)
(353960, 28, 28, 1)


In [8]:
print(y_train.shape)

(353960,)


In [9]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [10]:
EPOCHS = 50
LR = 1e-1
BS = 128



In [11]:
from sklearn.preprocessing import LabelBinarizer

le = LabelBinarizer()
labels = le.fit_transform(labels_total)
counts = labels.sum(axis=0)

classTotals = labels.sum(axis=0)
classWeight = {}
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]
print(len(classWeight))

36


In [17]:
input_t = tf.keras.Input(shape = (32,32,1))
res_model =tf.keras.applications.ResNet50(include_top=False, weights="imagenet")


In [13]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D,Conv2D

model = Sequential()
model.add(Conv2D(3,(3, 3),padding = "same",input_shape=(32,32,1)))
model.add(res_model)
model.add(Flatten())
model.add(Dense(len(le.classes_),kernel_regularizer=tf.keras.regularizers.l2(0.0001),activation='softmax'))


In [14]:
opt = tf.keras.optimizers.SGD(LR,decay = LR/EPOCHS) 
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 3)         30        
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 36)                73764     
Total params: 23,661,506
Trainable params: 23,608,386
Non-trainable params: 53,120
_________________________________________________________________


In [15]:
print(y_train.shape)


(353960,)


In [ ]:
history = model.fit(datagen.flow(X_train,y_train,batch_size = BS),
                   validation_data = (X_test,y_test),
                   steps_per_epoch = len(X_train)/BS,
                   epochs = EPOCHS,
                  class_weight = classWeight,
                   verbose = 1)

labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

#model evaluation

pred = model.predict(y_test,batch_size = BS)
print(classification_report(y_test.argmax(axis=1)),
                             pred.argmax(axis=1),
                             target_names = labelNames)

model.save("resNet50.h5")
